In [8]:
import pandas as pd

def csv_to_fasta(csv_file, output_fasta):
    df = pd.read_csv(csv_file)
    with open(output_fasta, 'w') as f:
        for _, row in df.iterrows():
            entry = row['Entry']
            sequence = row['Sequence']
            protien_name = row['Protein names']
            # f.write(f'>{entry}\n{sequence}\n')
            f.write(f'>{entry} | {protien_name}\n{sequence}\n')

csv_to_fasta('arch_retrieval.csv', 'arch.fasta')
csv_to_fasta('euk_retrieval.csv', 'euk.fasta')


In [ ]:
# makeblastdb -in arch.fasta -dbtype prot -out database/arch_db
# makeblastdb -in euk.fasta -dbtype prot -out database/euk_db


SyntaxError: invalid syntax (1132907889.py, line 1)

In [ ]:
import subprocess

def run_blast(query_fasta, db_name, output_file):
    cmd = [
        "blastp",
        "-query", query_fasta,
        "-db", db_name,
        "-out", output_file,
        "-outfmt", "6 qseqid sseqid evalue bitscore",
        "-evalue", "1e-3",
        "-max_target_seqs", "5"
    ]
    subprocess.run(cmd)

scoring_matrix = ["BLOSUM62", "BLOSUM80", "PAM30", "PAM70"]
run_blast("arch.fasta", "database/arch_db", "arch_blast_results.txt")
run_blast("euk.fasta", "database/euk_db", "euk_blast_results.txt")


In [11]:
from collections import defaultdict

def load_annotations(csv_file):
    df = pd.read_csv(csv_file)
    return dict(zip(df['Entry'], df['Protein names']))

# def load_blast_results(blast_file):
#     results = defaultdict(list)
#     with open(blast_file, 'r') as f:
#         for line in f:
#             qid, sid, *_ = line.strip().split()
#             if qid != sid:
#                 results[qid].append(sid)
#     return results

def load_blast_results(blast_file, top_k=None):
    results = defaultdict(list)

    with open(blast_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 4:
                continue
            qid, sid, evalue, bitscore = parts[0], parts[1], float(parts[2]), float(parts[3])
            if qid != sid:
                results[qid].append({
                    "sid": sid,
                    "bitscore": bitscore,
                    "evalue": evalue
                })

    for qid in results:
        results[qid].sort(key=lambda x: x["bitscore"], reverse=True)
        if top_k:
            results[qid] = results[qid][:top_k]

    return results

In [ ]:
# def compute_metrics_at_5(blast_results, annotations):
#     total_ap = 0
#     total_precision = 0
#     total_recall = 0
#     valid_queries = 0

#     # Create a reverse mapping: label → all entries with that label
#     label_to_entries = defaultdict(set)
#     for entry, label in annotations.items():
#         label_to_entries[label].add(entry)

#     for query, retrieved in blast_results.items():
#         if query not in annotations:
#             continue

#         query_label = annotations[query]
#         correct_set = label_to_entries[query_label] - {query}
#         relevant = [sid for sid in retrieved if sid in correct_set]

#         # Precision@5: correct / total retrieved
#         precision = len(relevant) / 5

#         # Recall@5: correct / all possible correct (excluding self)
#         recall = len(relevant) / len(correct_set) if correct_set else 0

#         # MAP@5
#         binary_relevance = [sid in correct_set for sid in retrieved]
#         precisions = [
#             sum(binary_relevance[:i+1]) / (i+1)
#             for i, rel in enumerate(binary_relevance) if rel
#         ]
#         ap = sum(precisions) / min(5, len(correct_set)) if precisions else 0

#         total_ap += ap
#         total_precision += precision
#         total_recall += recall
#         valid_queries += 1

#     if valid_queries == 0:
#         return 0, 0, 0

#     return (
#         total_ap / valid_queries,
#         total_precision / valid_queries,
#         total_recall / valid_queries
#     )


In [19]:
from collections import defaultdict

def compute_map_at_5(blast_results, annotations):
    label_to_entries = defaultdict(set)
    for entry, label in annotations.items():
        label_to_entries[label].add(entry)

    total_ap = 0
    valid_queries = 0

    for query, hits in blast_results.items():
        if query not in annotations:
            print(f"Query {query} not found in annotations.")
            continue

        query_label = annotations[query]
        true_positives = label_to_entries[query_label] - {query}
        if not true_positives:
            continue

        binary_relevance = [(hit["sid"] in true_positives) for hit in hits[:5]]
        if not any(binary_relevance):
            valid_queries += 1
            continue

        precisions = [
            sum(binary_relevance[:i+1]) / (i+1)
            for i, rel in enumerate(binary_relevance) if rel
        ]

        ap = sum(precisions) / min(5, len(true_positives))
        total_ap += ap
        valid_queries += 1

    return total_ap / valid_queries 


In [18]:
euk_annot = load_annotations("euk_retrieval.csv")
euk_blast = load_blast_results("euk_blast_results.txt")

map5 = compute_map_at_5(euk_blast, euk_annot)



print(f"MAP@5 for Eukaryotes: {map5:.4f}")
# print(f"Precision@5 for Eukaryotes: {p5:.4f}")
# print(f"Recall@5 for Eukaryotes: {r5:.4f}")


No true positives for query P96110.
No true positives for query Q88VW2.
No true positives for query P54386.
No true positives for query Q24YW4.
No true positives for query Q7N8D4.
No true positives for query Q8ABV5.
MAP@5 for Eukaryotes: 0.8007
